In [30]:
student_vle = pd.read_csv('studentVle.csv')  # The student interaction data
courses = pd.read_csv('courses.csv')  # The course length data


In [31]:
import pandas as pd

In [32]:
df = pd.merge(student_vle, courses, on=['code_module', 'code_presentation'], how='left')


In [33]:
df.shape

(10655280, 7)

In [34]:

duplicate_rows = df[df.duplicated(subset=['code_module', 'code_presentation', 'id_student', 'date'], keep=False)]
print(duplicate_rows)

         code_module code_presentation  id_student  id_site  date  sum_click  \
0                AAA             2013J       28400   546652   -10          4   
1                AAA             2013J       28400   546652   -10          1   
2                AAA             2013J       28400   546652   -10          1   
3                AAA             2013J       28400   546614   -10         11   
4                AAA             2013J       28400   546714   -10          1   
...              ...               ...         ...      ...   ...        ...   
10655266         GGG             2014J      688388   896939   269          7   
10655267         GGG             2014J      688388   896939   269          3   
10655277         GGG             2014J      654064   896943   269          3   
10655278         GGG             2014J      654064   896939   269          1   
10655279         GGG             2014J      654064   896939   269          1   

          module_presentation_length  


In [35]:
# Check for duplicates
duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")


Number of duplicate rows: 787170


In [36]:
# Remove duplicate rows
df = df.drop_duplicates()

# Verify after removing duplicates
print(f"Rows after removing duplicates: {df.shape[0]}")


Rows after removing duplicates: 9868110


In [37]:
# Display duplicate rows
duplicates = df[df.duplicated(keep=False)]
print(duplicates.head())


Empty DataFrame
Columns: [code_module, code_presentation, id_student, id_site, date, sum_click, module_presentation_length]
Index: []


In [38]:
# Check uniqueness of merge keys in courses.csv
is_unique = courses[['code_module', 'code_presentation']].duplicated().any()
if not is_unique:
    print("Merge keys are unique.")
else:
    print("Merge keys have duplicates!")


Merge keys are unique.


In [39]:
# Remove duplicates in courses.csv if necessary
courses = courses.drop_duplicates(subset=['code_module', 'code_presentation'])


In [40]:
print(f"Final number of rows: {df.shape[0]}")


Final number of rows: 9868110


In [41]:
df['percentage_completion'] = (df['date'] / df['module_presentation_length'] * 100).clip(upper=100)
df

,code_module,code_presentation,id_student,id_site,date,sum_click,module_presentation_length,percentage_completion
0,AAA,2013J,28400,546652,-10,4,268,-3.731343
1,AAA,2013J,28400,546652,-10,1,268,-3.731343
3,AAA,2013J,28400,546614,-10,11,268,-3.731343
4,AAA,2013J,28400,546714,-10,1,268,-3.731343
5,AAA,2013J,28400,546652,-10,8,268,-3.731343
...,...,...,...,...,...,...,...,...
10655274,GGG,2014J,679769,896943,269,2,269,100.000000
10655275,GGG,2014J,675811,896943,269,3,269,100.000000
10655276,GGG,2014J,675578,896943,269,1,269,100.000000
10655277,GGG,2014J,654064,896943,269,3,269,100.000000


In [42]:
# Define percentage ranges
percent_ranges = [20, 40, 60, 80, 100]

# Step 3: Initialize a results DataFrame
results = []

# Group by student and module-presentation
grouped = df.groupby(['code_module', 'code_presentation', 'id_student'])


In [43]:

for (module, presentation, student), group in grouped:
    student_result = {'code_module': module, 'code_presentation': presentation, 'id_student': student}
    for p in percent_ranges:
        # Filter data within the percentage range
        clicks = group[group['percentage_completion'] <= p]['sum_click']
        student_result[f'AC{p}%'] = clicks.mean() if not clicks.empty else 0
    results.append(student_result)


In [44]:

# Step 4: Convert results into a DataFrame
final_df = pd.DataFrame(results)


In [45]:
final_df

,code_module,code_presentation,id_student,AC20%,AC40%,AC60%,AC80%,AC100%
0,AAA,2013J,11391,5.988636,5.252101,5.338346,5.275168,4.802083
1,AAA,2013J,28400,3.793103,3.510204,3.286164,3.443069,3.403382
2,AAA,2013J,30268,3.611111,3.611111,3.611111,3.611111,3.611111
3,AAA,2013J,31604,3.381166,3.484375,3.373550,3.299814,3.306425
4,AAA,2013J,32885,3.953333,3.752577,3.484979,3.185053,3.020896
...,...,...,...,...,...,...,...,...
29223,GGG,2014J,2640965,2.235294,2.235294,2.235294,2.235294,2.235294
29224,GGG,2014J,2645731,2.291667,2.264706,4.307018,4.129353,3.976744
29225,GGG,2014J,2648187,1.545455,2.363636,2.750000,3.191011,3.095745
29226,GGG,2014J,2679821,4.352941,4.627119,4.627119,4.627119,4.627119
